# Offline RAG Evaluation

## Load Documents with IDS

In [16]:
import json
with open('documents-with-ids.json','rt') as f:
    docs_raw = json.load(f)

In [18]:
documents = []
for course_dict in docs_raw:
    documents.append(course_dict)

## Load Ground Truth Dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('ground_truth_dataset.csv')

In [4]:
df.head(5)

,question,course,document
0,When will the course start?,data-engineering-zoomcamp,c02e79ef
1,How do I subscribe to the course Google Calendar?,data-engineering-zoomcamp,c02e79ef
2,What should I do before the course starts?,data-engineering-zoomcamp,c02e79ef
3,How can I join the course Telegram channel?,data-engineering-zoomcamp,c02e79ef
4,Where should I register and join the channel i...,data-engineering-zoomcamp,c02e79ef


In [10]:
df = df[df['course']=='machine-learning-zoomcamp']

In [11]:
df.count()

question    1776
course      1776
document    1776
dtype: int64

In [13]:
ground_truth = df.to_dict(orient='records')

In [14]:
ground_truth[0]

{'question': 'What is the purpose of the document?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [23]:
doc_idx = {d['id']: d for d in documents}
print(doc_idx['0227b872']['text'])

Machine Learning Zoomcamp FAQ
The purpose of this document is to capture frequently asked technical questions.
We did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:
Data Engineering Zoomcamp FAQ
In the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo
work
